Before we start, I personally set goals.

1. Get fomatted answer (with retry)
2. tag with multiple document(uuid maybe)
3. test guardrails

In [13]:
import os
import dspy
from dspy.retrieve.weaviate_rm import WeaviateRM
import weaviate


weaviate_client = weaviate.Client("http://localhost:8080",
    additional_headers={
        "X-Openai-Api-Key": os.getenv("OPENAI_API_KEY"),
    }
)

llm = dspy.OpenAI(model="gpt-3.5-turbo", max_tokens=4000, model_type="chat")
weaviate_rm = WeaviateRM("WeaviateBlogChunk", weaviate_client=weaviate_client)
dspy.settings.configure(lm=llm, rm=weaviate_rm)

In [14]:
schema = {
  "classes": [
    {
      "class": "WeaviateBlogChunk",
      "vectorizer": "text2vec-openai",
      "properties": [
        {
          "name": "content",
          "dataType": ["text"],
        },
        {
          "name": "author",
          "dataType": ["text"],
        }
      ],
      "moduleConfig": {
        "text2vec-openai": {
          "model": "text-embedding-3-large"
        }
      }
    }
  ]
}

# 스키마 생성
weaviate_client.schema.create(schema)

c:\anaconda\envs\dspy\Lib\site-packages\pygments\regexopt.py:78: ResourceWarning: unclosed <socket.socket fd=580, family=23, type=1, proto=0, laddr=('::', 64179, 0, 0)>
  for group in groupby(strings, lambda s: s[0] == first[0])) \


UnexpectedStatusCodeException: Create class! Unexpected status code: 422, with response body: {'error': [{'message': 'class name "WeaviateBlogChunk" already exists'}]}.

In [15]:
# Chunk Blogs

import os
import re

def chunk_list(lst, chunk_size):
    """Break a list into chunks of the specified size."""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

def split_into_sentences(text):
    """Split text into sentences using regular expressions."""
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return [sentence.strip() for sentence in sentences if sentence.strip()]

def read_and_chunk_index_files(main_folder_path):
    """Read index.md files from subfolders, split into sentences, and chunk every 5 sentences."""
    blog_chunks = []
    for folder_name in os.listdir(main_folder_path):
        subfolder_path = os.path.join(main_folder_path, folder_name)
        if os.path.isdir(subfolder_path):
            index_file_path = os.path.join(subfolder_path, 'index.mdx')
            if os.path.isfile(index_file_path):
                with open(index_file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    sentences = split_into_sentences(content)
                    sentence_chunks = chunk_list(sentences, 5)
                    sentence_chunks = [' '.join(chunk) for chunk in sentence_chunks]
                    blog_chunks.extend(sentence_chunks)
    return blog_chunks

# Example usage
main_folder_path = './blog'
blog_chunks = read_and_chunk_index_files(main_folder_path)


print(f"There are {len(blog_chunks)} chunks in the main folder path.")

There are 1182 chunks in the main folder path.


In [16]:
import weaviate
for blog_chunk in blog_chunks:
    blog_object = {
        "content": blog_chunk,
    }

    try:
        response = weaviate_client.data_object.create(
            data_object=blog_object,
            class_name="WeaviateBlogChunk"
        )
        print(f"삽입 성공, UUID: {response}")
    except Exception as e:
        print(f"데이터 삽입 중 오류 발생: {e}")

삽입 성공, UUID: 4fab49c7-e0b6-42c5-abab-0c7c0c214b2a
삽입 성공, UUID: a8e69d56-d099-41ec-93d7-ed915987b8a2
삽입 성공, UUID: 4e060d15-c36f-4ad3-8e98-870d6d0a9ad6
삽입 성공, UUID: 9ca4d0c3-6b05-4ef7-b656-4f3eb51a9417
삽입 성공, UUID: 78505c23-8884-45cb-a2af-2c5296becf43
삽입 성공, UUID: 0577f378-0964-479e-927d-8c2973fb1e92
삽입 성공, UUID: 535624c4-c2d8-4035-abad-fa29fb10d1ac
삽입 성공, UUID: 41e02cfc-662e-4100-bc75-1a3b3410bebc
삽입 성공, UUID: 60605a9e-8f6e-4e83-854f-9760838807a3
삽입 성공, UUID: 7a82b920-41dc-4582-85e1-5580954e0902
삽입 성공, UUID: 0c3921eb-d810-41e7-bd1a-2b6adc2d22ef
삽입 성공, UUID: 05ad6775-afde-48b0-b823-778026bedd40
삽입 성공, UUID: 35c1d2dd-777e-48ce-9c1b-d27a0f06abff
삽입 성공, UUID: 4fbdc7a9-f26a-4e7c-96a1-5128675a0ff7
삽입 성공, UUID: 5dec7d1f-3c0a-4fa6-bcfa-597741e4e24e
삽입 성공, UUID: 59262854-7fcd-4337-aa35-9ed9257ab9e3
삽입 성공, UUID: 3b0e17d2-ca7b-4677-bf49-764dfc9ac338
삽입 성공, UUID: 4a4f793c-2c3b-4b6a-9c77-d254fe46ecda
삽입 성공, UUID: fcd5592e-07dc-4e88-8ad0-1906e1c6abb2
삽입 성공, UUID: dbe570cb-c38b-480e-8f87-2594f556fdd3


In [17]:
def format_weaviate_context(weaviateRM_output):
    weaviateRM_output = [d['long_text'] for d in weaviateRM_output]
    weaviateRM_output = "".join(weaviateRM_output)
    return weaviateRM_output

In [37]:
from dspy.functional import TypedPredictor
import pydantic
from typing import List

class Topic(pydantic.BaseModel):
    topic: str
    topic_description: str

class Topics(pydantic.BaseModel):
    topics: List[Topic]

class TypedQuestion2BlogOutline(dspy.Signature):
    """Your task is to write a Weaviate blog post that will help answer the given question.\nPlease use the contexts from a web search and published Weaviate blog posts to evaluate the structure of the blog post."""
    
    question: str = dspy.InputField()
    blog_context: str = dspy.InputField()
    blog_outline: Topics = dspy.OutputField(desc="A list of topics the blog will cover. IMPORTANT!! This must follow a comma separated list of values!")

In [38]:
import functools

class TypedBlog2Outline(dspy.Module):
    def __init__(self, weaviate_rm):
        self.question_to_blog_outline = dspy.functional.TypedPredictor(TypedQuestion2BlogOutline)
        self.weaviate_rm = weaviate_rm

    def forward(self, question):
        blog_contexts = self.weaviate_rm(question, k=3)
        blog_contexts= format_weaviate_context(blog_contexts)
        question_to_blog_outline_outputs = self.question_to_blog_outline(question=question, blog_context=blog_contexts)
        return question_to_blog_outline_outputs.blog_outline

                                                                         
blog2outline = TypedBlog2Outline(weaviate_rm)
        
toy_question = "What are cross encoders?"


print(f"\033[91mResult for gpt-3.5-turbo\n")
print(f"\033[0m{blog2outline(question=toy_question)} \n")

Result for gpt-3.5-turbo

topics=[Topic(topic='Introduction to Cross Encoders', topic_description='Explanation of what cross encoders are and how they are used for content-based re-ranking'), Topic(topic='Types of Cross Encoders', topic_description='Overview of different types of cross encoders available, such as sentence transformers'), Topic(topic='Functionality of Cross-Encoder Models', topic_description='Explanation of how cross-encoder models work, including the input-output mechanism and similarity calculation'), Topic(topic='Efficiency Challenges of Cross-Encoders', topic_description='Discussion on the inefficiency of using cross-encoder models for large-scale semantic search applications'), Topic(topic='Combining Bi-Encoders and Cross-Encoders', topic_description='Exploration of the benefits of combining bi-encoders and cross-encoders for improved search performance')] 

